# 智能体框架

## 基本功能

### log + call

In [1]:
from illufly.agent import FakeLLM
from illufly.io import log

log(FakeLLM(), "hi")

这是一个模拟调用!



'这是一个模拟调用!'

### alog + async_call

In [2]:
from illufly.agent import FakeLLM
from illufly.io import alog

await alog(FakeLLM(), "hi", sleep=1)

这是一个模拟调用!



'这是一个模拟调用!'

### 线程池

In [3]:
FakeLLM.monitor_executors()

{'FAKE_LLM': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0}}

## 智能体：直调大模型

### 对话

In [4]:
from illufly.agent import ChatQwen
from illufly.io import log

qwen = ChatQwen()
log(qwen, "你能帮我写一首关于兔子做梦的四句儿歌?")

小白兔，梦中跳，  
胡萝卜，满天飘。  
月亮船，带它逛，  
醒来笑，梦真妙。



'小白兔，梦中跳，  \n胡萝卜，满天飘。  \n月亮船，带它逛，  \n醒来笑，梦真妙。'

In [5]:
qwen.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小白兔，梦中跳，  \n胡萝卜，满天飘。  \n月亮船，带它逛，  \n醒来笑，梦真妙。'}]

### 使用系统提示语

In [6]:
from illufly.agent import ChatQwen
from illufly.io import log

qwen = ChatQwen(prompt="你是一个专门写儿歌的作家，请根据我的提示写作。")
log(qwen, "来一首关于兔子的，四句")

小白兔，蹦蹦跳，  
耳朵长，尾巴小。  
爱吃萝卜和青菜，  
快乐生活在林梢。



'小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'

In [7]:
qwen.memory

[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant',
  'content': '小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'}]

In [8]:
log(qwen, "换成两条小鱼")

两条小鱼，游啊游，  
水中穿梭，乐悠悠。  
摇摇尾巴，吐泡泡，  
大海深处是故乡。



'两条小鱼，游啊游，  \n水中穿梭，乐悠悠。  \n摇摇尾巴，吐泡泡，  \n大海深处是故乡。'

In [9]:
qwen.memory

[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant',
  'content': '小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'},
 {'role': 'user', 'content': '换成两条小鱼'},
 {'role': 'assistant',
  'content': '两条小鱼，游啊游，  \n水中穿梭，乐悠悠。  \n摇摇尾巴，吐泡泡，  \n大海深处是故乡。'}]

## 智能体：使用工具

### 返回工具提示

In [10]:
from illufly.tools import tool, convert_to_openai_tool
from illufly.agent import ChatQwen
from illufly.io import log
import json

@tool
def get_current_weather(location: str=None):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

In [11]:
log(ChatQwen(), "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])
# log(ChatQwen(), "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

'{"0": {"index": 0, "id": "call_6d87845fd30b41208b9c83", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}'

### 执行工具回调

In [12]:
qwen = ChatQwen(
    tools=[convert_to_openai_tool(get_current_weather)],
    toolkits=[get_current_weather]
)

log(qwen, "今天广州可以晒被子吗？")

广州今天是晴天。 

今天广州是晴天，非常适合晒被子。可以放心地把被子拿出来晾晒。



'{"0": {"index": 0, "id": "call_99a3fa145c734c9891902a", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}今天广州是晴天，非常适合晒被子。可以放心地把被子拿出来晾晒。'

## 智能体团队：执行管道

### 从聊天开始的管道 [Chat, Chat, Chat]

In [4]:
from illufly.agent import ChatQwen, Pipe
from illufly.io import log

pipe = Pipe(
    ChatQwen(prompt="我是一个儿童作家，擅长写儿歌。"),
    ChatQwen(prompt="请你帮我评价文章特色，两句话即可"),
    ChatQwen(prompt="请针对我的写作成果打一个分数，给出一句话的打分点，最终给出1分至5分")
)

log(pipe, "你能帮我写一首关于兔子做梦的？四句即可。")

[AGENT] >>> Node 1: 我是一个儿童作家，擅长写儿歌。
小白兔，梦中跳，月亮船上摇啊摇。
胡萝卜，变成桥，梦里世界真奇妙。

[AGENT] >>> Node 2: 请你帮我评价文章特色，两句话即可
这首短文充满了童趣和想象力，通过"小白兔在月亮船上摇晃"和"胡萝卜变成桥"的奇幻意象，展现了梦境的奇妙与无尽创意，语言简洁，富有诗意，非常适合儿童阅读，激发他们的想象空间。

[AGENT] >>> Node 3: 请针对我的写作成果打一个分数，给出一句话的打分点，最终给出1分至5分
4分。打分点在于作品成功营造了富有童趣和想象力的氛围，语言表达既简洁又有诗意，非常适合儿童阅读，但在内容深度或情节构建上还有提升空间。



'4分。打分点在于作品成功营造了富有童趣和想象力的氛围，语言表达既简洁又有诗意，非常适合儿童阅读，但在内容深度或情节构建上还有提升空间。'

In [6]:
pipe.runnables[0].memory

[{'role': 'system', 'content': '我是一个儿童作家，擅长写儿歌。'},
 {'role': 'user', 'content': '你能帮我写一首关于兔子做梦的？四句即可。'},
 {'role': 'assistant', 'content': '小白兔，梦中跳，月亮船上摇啊摇。\n胡萝卜，变成桥，梦里世界真奇妙。'}]

### 从提示语构造开始的管道 [Template, Chat, Chat]

In [7]:
from illufly.agent import ChatQwen, Pipe, Template
from illufly.io import log, alog

In [8]:
t = Template("IDEA")
log(t, {"task": "请你帮我写一首4行儿歌"})
t.output

'请开始'

In [9]:
pipe = Pipe(
    Template("IDEA"),
    ChatQwen(),
    ChatQwen(prompt="请帮我对作品进行评价，找出三个缺点，一句话总结。")
)

log(pipe, {"task": "请你帮我写一首4行儿歌"})

[AGENT] >>> Node 1: Template
[AGENT] >>> Node 2: ChatQwen
小蜜蜂，嗡嗡嗡，  
飞到花丛中，  
采花蜜，忙不停，  
甜甜蜜带回家。

[AGENT] >>> Node 3: 请帮我对作品进行评价，找出三个缺点，一句话总结。
1. **内容单一**：诗作围绕蜜蜂采蜜的活动展开，主题较为单一，缺乏情节的起伏或深度探索。
2. **形象刻画浅显**：对小蜜蜂的描述仅限于基本的动作与声音，未能深入其生态习性或情感层面，使角色显得较为平面。
3. **语言简单重复**：使用了较多的叠词和简单句式，虽符合儿童诗歌的特点，但缺乏变化和想象力的词汇运用，可能不足以吸引更广泛年龄段读者的兴趣。

**一句话总结**：这首诗作为儿童启蒙读物简洁明快，但因内容单一、形象刻画浅显及语言简单重复，略显缺乏深度与吸引力。



'1. **内容单一**：诗作围绕蜜蜂采蜜的活动展开，主题较为单一，缺乏情节的起伏或深度探索。\n2. **形象刻画浅显**：对小蜜蜂的描述仅限于基本的动作与声音，未能深入其生态习性或情感层面，使角色显得较为平面。\n3. **语言简单重复**：使用了较多的叠词和简单句式，虽符合儿童诗歌的特点，但缺乏变化和想象力的词汇运用，可能不足以吸引更广泛年龄段读者的兴趣。\n\n**一句话总结**：这首诗作为儿童启蒙读物简洁明快，但因内容单一、形象刻画浅显及语言简单重复，略显缺乏深度与吸引力。'

In [10]:
pipe.memory

[{'role': 'user', 'content': '节点 <0> 正在处理任务...'},
 {'role': 'assistant', 'content': '请开始'},
 {'role': 'user', 'content': '节点 <1> 正在处理任务...'},
 {'role': 'assistant', 'content': '小蜜蜂，嗡嗡嗡，  \n飞到花丛中，  \n采花蜜，忙不停，  \n甜甜蜜带回家。'},
 {'role': 'user', 'content': '节点 <2> 正在处理任务...'},
 {'role': 'assistant',
  'content': '1. **内容单一**：诗作围绕蜜蜂采蜜的活动展开，主题较为单一，缺乏情节的起伏或深度探索。\n2. **形象刻画浅显**：对小蜜蜂的描述仅限于基本的动作与声音，未能深入其生态习性或情感层面，使角色显得较为平面。\n3. **语言简单重复**：使用了较多的叠词和简单句式，虽符合儿童诗歌的特点，但缺乏变化和想象力的词汇运用，可能不足以吸引更广泛年龄段读者的兴趣。\n\n**一句话总结**：这首诗作为儿童启蒙读物简洁明快，但因内容单一、形象刻画浅显及语言简单重复，略显缺乏深度与吸引力。'}]

In [12]:
pipe.runnables[0].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请你帮我写一首4行儿歌\n'},
 {'role': 'user', 'content': '请开始'}]

In [13]:
pipe.runnables[1].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请你帮我写一首4行儿歌\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '小蜜蜂，嗡嗡嗡，  \n飞到花丛中，  \n采花蜜，忙不停，  \n甜甜蜜带回家。'}]

In [14]:
log(pipe.runnables[1], "找出三个优点吧")

1. **简洁明了**：这首儿歌语言简练，易于儿童理解和记忆。
2. **节奏感强**：通过重复的音节和韵脚，如“嗡嗡嗡”和“中”，创造了欢快的节奏，吸引孩子兴趣。
3. **教育意义**：在歌唱的同时，向儿童传递了自然界中小蜜蜂勤劳采蜜的知识，培养孩子对自然的好奇心和尊重劳动的价值观。



'1. **简洁明了**：这首儿歌语言简练，易于儿童理解和记忆。\n2. **节奏感强**：通过重复的音节和韵脚，如“嗡嗡嗡”和“中”，创造了欢快的节奏，吸引孩子兴趣。\n3. **教育意义**：在歌唱的同时，向儿童传递了自然界中小蜜蜂勤劳采蜜的知识，培养孩子对自然的好奇心和尊重劳动的价值观。'

## 智能体团队：从提纲扩写

### 构建提纲

In [15]:
from illufly.agent import ChatQwen, Pipe, FromOutline, Template
from illufly.io import log, alog

outline = Pipe(
    Template("OUTLINE"),
    ChatQwen()
)

In [16]:
log(outline, {"task": "写一首两段儿歌，每段20个字即可，策划简单一点"})

[AGENT] >>> Node 1: Template
[AGENT] >>> Node 2: ChatQwen
# 儿歌：小星星的梦

## 第一段：星空奇妙夜
<OUTLINE>
- 描述夜晚星空的美丽景象，提及小星星
- 引入小星星梦想飞翔的念头，充满好奇与期待
- 字数：约20字
</OUTLINE>

## 第二段：晨光中的舞者
<OUTLINE>
- 描述随着黎明到来，小星星与月亮道别的温馨场景
- 小星星化作流星，短暂而灿烂地“舞动”于天际
- 字数：约20字
</OUTLINE>



'# 儿歌：小星星的梦\n\n## 第一段：星空奇妙夜\n<OUTLINE>\n- 描述夜晚星空的美丽景象，提及小星星\n- 引入小星星梦想飞翔的念头，充满好奇与期待\n- 字数：约20字\n</OUTLINE>\n\n## 第二段：晨光中的舞者\n<OUTLINE>\n- 描述随着黎明到来，小星星与月亮道别的温馨场景\n- 小星星化作流星，短暂而灿烂地“舞动”于天际\n- 字数：约20字\n</OUTLINE>'

### 提纲 + 扩写

In [17]:
from illufly.agent import ChatQwen, Pipe, FromOutline, Template
from illufly.io import log, alog

writer = Pipe(
    Template("OUTLINE"),
    ChatQwen(),
    FromOutline(ChatQwen())
)

log(writer, {"task": "写一首两段儿歌，每段20个字即可，策划简单一点"})

[AGENT] >>> Node 1: Template
[AGENT] >>> Node 2: ChatQwen
# 儿歌：小星星的夜游

## 第一段：星星醒来
<OUTLINE>
扩写要求：
- 描述夜晚降临，星星出现在天空的情景
- 引入主角小星星，它眨着眼睛好奇世界
- 预估字数：20字
</OUTLINE>

## 第二段：月亮朋友
<OUTLINE>
扩写要求：
- 介绍小星星遇到月亮，它们在夜空玩耍
- 表达友谊与快乐的氛围
- 预估字数：20字
</OUTLINE>

[AGENT] >>> Node 3: FromOutline
[AGENT] 执行扩写任务 <0169-399-003>：
扩写要求：
- 描述夜晚降临，星星出现在天空的情景
- 引入主角小星星，它眨着眼睛好奇世界
- 预估字数：20字
[AGENT] >>> Node 1: Template
[AGENT] >>> Node 2: ChatQwen
夜幕轻垂，万籁俱寂，星空渐渐亮起眼眸。小星星闪耀登场，好奇地眨，探秘夜的温柔。

[AGENT] 执行扩写任务 <0169-572-006>：
扩写要求：
- 介绍小星星遇到月亮，它们在夜空玩耍
- 表达友谊与快乐的氛围
- 预估字数：20字
[AGENT] >>> Node 1: Template
[AGENT] >>> Node 2: ChatQwen
小星星遇见了月亮姐姐，手拉手舞动在夜空，欢笑声响彻云霄。



'小星星遇见了月亮姐姐，手拉手舞动在夜空，欢笑声响彻云霄。'

In [18]:
writer.runnables[1].output

'# 儿歌：小星星的夜游\n\n## 第一段：星星醒来\n<OUTLINE>\n扩写要求：\n- 描述夜晚降临，星星出现在天空的情景\n- 引入主角小星星，它眨着眼睛好奇世界\n- 预估字数：20字\n</OUTLINE>\n\n## 第二段：月亮朋友\n<OUTLINE>\n扩写要求：\n- 介绍小星星遇到月亮，它们在夜空玩耍\n- 表达友谊与快乐的氛围\n- 预估字数：20字\n</OUTLINE>'

In [19]:
print(writer.output)

# 儿歌：小星星的夜游


## 第一段：星星醒来

夜幕轻垂，万籁俱寂，星空渐渐亮起眼眸。小星星闪耀登场，好奇地眨，探秘夜的温柔。


## 第二段：月亮朋友

小星星遇见了月亮姐姐，手拉手舞动在夜空，欢笑声响彻云霄。




In [20]:
writer.runnables

(<illufly.agent.template.Template at 0x11ee69b10>,
 <illufly.agent.team.from_outline.FromOutline at 0x11ee3ae00>)